In [39]:
from datetime import datetime

data = []

with open('_chat.txt') as file:
    for i, line in enumerate(file.read().split('[')):
        line = line.strip('\n')
        if line != '':
            timestamp, rest = line.split(']')
            timestamp = datetime.strptime(timestamp, "%d-%m-%Y %H:%M:%S")
            sender, message = rest.split(': ', maxsplit=1)
            message = message.strip('\u200e')
            data.append({
                'timestamp': timestamp,
                'sender': sender,
                'message': message,
            })
            # if message == 'sticker weggelaten':
            #     print(message)
            # print(i, message)

In [40]:
import pandas as pd

pd.DataFrame(data)

ModuleNotFoundError: No module named 'pandas'